In [1]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)

In [2]:
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [3]:
def get_data():
    def norm(x):
        std = np.std(x ,ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/1 Мира' # positive class

    PATHS = li.util.find_files(PATH_0)[:1000] + li.util.find_files(PATH_1)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    SOUNDS = []
    for path in PATHS:
        SOUNDS.append(Audio(path))
        print('Loading {:.1f}%'.format(len(SOUNDS) / N * 100))
        ipd.clear_output(wait=True)
    DATA = []
    LABELS = []

    for sound in SOUNDS:
        DATA.append(norm(sound.e_parts))
        LABELS.append(sound.label)

    return (np.array(DATA), np.array(LABELS))

In [4]:
data, labels = get_data()

Loading 100.0%


In [88]:
test_data, test_labels = data[8000:], labels[8000:]
data, labels = data[:8000], labels[:8000]

In [5]:
data.shape, labels.shape

((1070, 20), (1070,))

In [48]:
BATCH_SIZE = 1081

dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 20), (None,)), types: (tf.float64, tf.int32)>

In [6]:
BATCH_SIZE = 1070
test_data = np.reshape(data, [1070, 1, 20])

dataset = tf.data.Dataset.from_tensor_slices((test_data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 1, 20), (None,)), types: (tf.float64, tf.int32)>

In [188]:
data = np.reshape(data, (10070, 20))

In [211]:
labels = np.reshape(labels, (10070,))

In [44]:
np.shape(data)

(1070, 20)

<h1>Создание модели нейронной сети</h1>

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 20,)),
    tf.keras.layers.GRU(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), reset_after=False),
    # tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(50, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(25, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(12, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(2, activation='softmax')
], name='Mira_GRU')

model.summary()

Model: "Mira_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 20)                2460      
_________________________________________________________________
dense (Dense)                (None, 2)                 42        
Total params: 2,502
Trainable params: 2,502
Non-trainable params: 0
_________________________________________________________________


PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

<h1>Обучение</h1>

In [16]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=1000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

Epoch 000 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9440385103225708, Precision: 0.9779969453811646
Epoch 010 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.944100558757782, Precision: 0.9780222177505493
Epoch 020 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9441624879837036, Precision: 0.9780474305152893
Epoch 030 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9442242383956909, Precision: 0.9780725836753845
Epoch 040 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9442858695983887, Precision: 0.978097677230835
Epoch 050 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9443473815917969, Precision: 0.9781227111816406
Epoch 060 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9444087147712708, Precision: 0.9781476855278015
Epoch 070 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9444699883460999, Precision: 0.9781726002693176
Epoch 080 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.944531083106994

In [17]:
model.save('models\{}-v1.1.h5'.format(model.name))

In [8]:
model = tf.keras.models.load_model(r'models\Mira-v1.0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [25]:
with tf.device("CPU"):
    predictions = tft.predict_classes(model=model, features=test_data)

In [26]:
n = 0

for i in range(len(labels)):
    n += int(labels[i] == predictions[i])

print(n, n / len(labels) * 100)

1072 100.0


In [27]:
predictions[60:70]

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [28]:
predictions[:15], labels[:15]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [29]:
model(data[:15])

InvalidArgumentError: transpose expects a vector of size 2. But input(1) is a vector of size 3 [Op:Transpose]

In [30]:
labels[60:70]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [72]:
model(np.reshape(data[0], (1, 1, 20)))

UnknownError: Fail to find the dnn implementation. [Op:CudnnRNN]

In [88]:
test_data[0].shape

(1, 20)

In [89]:
data[0].shape

(20,)

In [107]:
test_model = tf.keras.models.clone_model(model)

In [32]:
model(test_data[50:70])

UnknownError: Fail to find the dnn implementation. [Op:CudnnRNN]

'Mira GRU'